In [ ]:
import numpy as np
import pandas as pd
import os
import glob
from tqdm.notebook import tqdm
from PIL import Image

In [ ]:
os.listdir('/content/drive/MyDrive/1000images')

['0.0.Normal',
 '1.1.DR3',
 '10.0.Possible glaucoma',
 '1.0.DR2',
 '11.Severe hypertensive retinopathy',
 '0.2.Large optic cup',
 '10.1.Optic atrophy',
 '0.3.DR1',
 '0.1.Tessellated fundus',
 '2.0.BRVO',
 '19.Fundus neoplasm',
 '15.1.Bietti crystalline dystrophy',
 '18.Vitreous particles',
 '14.Congenital disc abnormality',
 '13.Dragged Disc',
 '15.0.Retinitis pigmentosa',
 '12.Disc swelling and elevation',
 '17.Myelinated nerve fiber',
 '16.Peripheral retinal degeneration and break',
 '28.Silicon oil in eye',
 '24.Chorioretinal atrophy-coloboma',
 '26.Fibrosis',
 '21.Yellow-white spots-flecks',
 '25.Preretinal hemorrhage',
 '22.Cotton-wool spots',
 '27.Laser Spots',
 '2.1.CRVO',
 '20.Massive hard exudates',
 '23.Vessel tortuosity',
 '3.RAO',
 '9.Pathological myopia',
 '7.ERM',
 '4.Rhegmatogenous RD',
 '5.0.CSCR',
 '6.Maculopathy',
 '29.1.Blur fundus with suspected PDR',
 '8.MH',
 '29.0.Blur fundus without PDR',
 '5.1.VKH disease']

In [ ]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
import torch.optim as optim

device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
class custom_dataset(Dataset):
    def __init__(self,root_dir,transform=None):

        self.data=[]
        self.transform=transform

        for img_path in tqdm(glob.glob(root_dir+"/*/**")):
            class_name=img_path.split("/")[-2]
            self.data.append([img_path,class_name])

        self.class_map={}
        for index,item in enumerate(os.listdir(root_dir)):
             self.class_map[item]=index
        print(f"Total Classes:{len(self.class_map)}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        img_path,class_name=self.data[idx]
        img=Image.open(img_path)
        class_id=self.class_map[class_name]
        class_id=torch.tensor(class_id)

        if self.transform:
            img=self.transform(img)

        return img,class_id

In [ ]:
root_dir=r'/content/drive/MyDrive/1000images (1)'

In [ ]:
def create_transforms(normalize=False,mean=[0,0,0],std=[1,1,1]):
    if normalize:
        my_transforms=transforms.Compose([
            transforms.Resize((224,224)),
#             transforms.ColorJitter(brightness=0.3,saturation=0.5,contrast=0.7,),
#             transforms.RandomRotation(degrees=33),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean,std=std)
        ])

    else:
         my_transforms=transforms.Compose([
            transforms.Resize((512,512)),
#             transforms.ColorJitter(brightness=0.3,saturation=0.5,contrast=0.7,p=0.57),
#             transforms.RandomRotation(degrees=33),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()])


    return my_transforms

In [ ]:
BS=8
num_classes=39

In [ ]:
my_transforms=create_transforms(normalize=False)
dataset=custom_dataset(root_dir,my_transforms)
print(len(dataset))

train_set, val_set=torch.utils.data.random_split(dataset,[800,200],generator=torch.Generator().manual_seed(7))
train_loader=DataLoader(train_set,batch_size=BS,shuffle=True)
val_loader=DataLoader(val_set,batch_size=BS,shuffle=True)

  0%|          | 0/1000 [00:00<?, ?it/s]

Total Classes:40
1000


In [ ]:
def get_mean_std(loader):
    #var=E[x^2]-(E[x])^2
    channels_sum, channels_squared_sum,num_batches=0,0,0
    for data,_ in tqdm(loader):
        channels_sum+=torch.mean(data,dim=[0,2,3]) # we dont want to a singuar mean for al 3 channels (in case of RGB)
        channels_squared_sum+=torch.mean(data**2,dim=[0,2,3])
        num_batches+=1
    mean=channels_sum/num_batches
    std=(channels_squared_sum/num_batches-mean**2)**0.5

    return mean, std

In [ ]:
mean,std=get_mean_std(train_loader)
print(mean, std)

  0%|          | 0/100 [00:00<?, ?it/s]

tensor([0.4621, 0.2672, 0.1061]) tensor([0.2384, 0.1514, 0.0813])


In [ ]:
#Since these are medical images (differenct from Imagenet data) I'll use the calculated mean, std
my_transforms=create_transforms(normalize=True,mean=mean,std = std)
dataset=custom_dataset(root_dir,my_transforms)
print(len(dataset))

train_set, val_set=torch.utils.data.random_split(dataset,[800,200],generator=torch.Generator().manual_seed(7))
train_loader=DataLoader(train_set,batch_size=BS,shuffle=True)
val_loader=DataLoader(val_set,batch_size=BS,shuffle=True)

  0%|          | 0/1000 [00:00<?, ?it/s]

Total Classes:40
1000


vgg model

In [ ]:
vgg_model=torchvision.models.vgg16(pretrained=True)
print(vgg_model)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
vgg_model=torchvision.models.vgg16(pretrained=True)

for param in vgg_model.parameters():
    param.requires_grad=False

class Identity(nn.Module):
    def __init__(self):
        super().__init__()


    def forward(self,x):
        return x

# vgg_model.avgpool=Identity()
vgg_model.classifier=nn.Sequential(
    nn.Linear(25088,2048),
    nn.ReLU(),
    nn.Dropout(p=0.37),
    nn.Linear(2048,1024),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(1024,num_classes)
)

vgg_model.to(device)

# model.features[30]=nn.AdaptiveAvgPool2d((16,16))

# print(model.features)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
EPOCHS=30
LR=1e-3

In [ ]:
def train_model(model):

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=LR)
    scheduler=optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9,verbose=True)

    for epoch in range(EPOCHS):
        losses=[]
        print(f"Epoch {epoch+1}/{EPOCHS}:")
        loop=tqdm(enumerate(train_loader),total=len(train_loader))
        for batch_idx,(data,targets) in loop:
            data=data.to(device)
            targets=targets.to(device)

            #forward
            scores=model(data)
            loss=criterion(scores,targets)

            losses.append(loss.item())

            #backward
            optimizer.zero_grad()
            loss.backward()

            #gradient descent/adam step
            optimizer.step()
        mean_loss=sum(losses)/len(losses)
        scheduler.step()

        print(f"Loss at Epoch {epoch+1}:\t{mean_loss:.5f}\n")

In [ ]:
def check_accuracy(loader, model):

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()

In [ ]:

train_model(vgg_model)

Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 1/30:


  0%|          | 0/100 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 1.0000e-03.
Loss at Epoch 1:	3.38299

Epoch 2/30:


  0%|          | 0/100 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 9.0000e-04.
Loss at Epoch 2:	2.39208

Epoch 3/30:


...

Adjusting learning rate of group 0 to 2.2877e-04.
Loss at Epoch 28:	0.22773

Epoch 29/30:


  0%|          | 0/100 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 2.2877e-04.
Loss at Epoch 29:	0.15853

Epoch 30/30:


  0%|          | 0/100 [00:00<?, ?it/s]

Adjusting learning rate of group 0 to 2.0589e-04.
Loss at Epoch 30:	0.18076



In [ ]:
print("Training accuracy:",end='\t')
check_accuracy(train_loader, vgg_model)
print("Validation accuracy:",end='\t')
check_accuracy(val_loader, vgg_model)

Training accuracy:	

  0%|          | 0/100 [00:00<?, ?it/s]

Got 800 / 800 with accuracy 100.00
Validation accuracy:	

  0%|          | 0/25 [00:00<?, ?it/s]

Got 146 / 200 with accuracy 73.00
